# Text analyze with ML

## Import modules

In [1]:
# imports for reading and writing (input & output) files:
import pandas as pd
import os

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from collections import Counter

import re   # regular expressions

In [2]:
# show several prints in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Reading input files
##### The data includes column with short stories in hebrew, and column with the writer's gender (the target)

In [3]:
# Train set
train_filename = '.' + os.sep + 'input' + os.sep + 'annotated_corpus_for_train.xlsx'
df_train = pd.read_excel(train_filename, 'corpus', index_col=None, na_values=['NA'])

# Untaged test set
test_filename  = '.' + os.sep + 'input' + os.sep + 'corpus_for_test.xlsx'
df_test  = pd.read_excel(test_filename,  'corpus', index_col=None, na_values=['NA'])

# Taged test set
my_filename  = '.' + os.sep + 'input' + os.sep + 'taged_test.xlsx'
my_test  = pd.read_excel(my_filename,  'corpus', index_col=None, na_values=['NA'])

In [4]:
df_train.head(5)

,story,gender
0,"בוקר אחד קמתי סהרורי יצאתי מהמיטה קצת מטושטש ,...",m
1,לחבר שלי היה יום הולדת וחיפשנו מה אפשר לעשות ל...,m
2,"השנה האחרונה הייתה שנת קורונה, שנה לא פשוטה בק...",m
3,"לפני כחצי שנה עברתי לגור בצפון עם בת זוגתי, עב...",m
4,"יום חמישי רגיל, תמיד מתחיל לעבור טיפה מאוחר יו...",m


In [5]:
df_test.head(5)

,test_example_id,story
0,0,כחלק ממסגרת ההתנדבות שלי במגלה אני הולך לפעמיי...
1,1,"לפני שנה החלטתי שאני רוצה להיות טייס, התחלתי ל..."
2,2,"בתקופת הקורונה של תחילת החיסונים נגד קורונה, א..."
3,3,כפי שכולם מכירים או שמעו מחברים עולם הדייטים ה...
4,4,"אחת החוויות שהכי זכורות לי, זו החוויה בפרו בטי..."


## Text manipulations

In [6]:
for i in df_train.index:
    # replace " character with non-char (for saving initials)
    df_train.iloc[i, 0] = re.sub(r'"', '', str(df_train.iloc[i, 0]))
    # replace special characters with space
    df_train.iloc[i, 0] = re.sub(r'\W', ' ', str(df_train.iloc[i, 0]))
    # replace numbers with space
    df_train.iloc[i, 0] = re.sub(r'\d', ' ', str(df_train.iloc[i, 0]))
    # replace one-letter word with space
    df_train.iloc[i, 0] = re.sub(r'\s\w\s', ' ', df_train.iloc[i, 0])
    # remove 'ו' at the beginning of words
    df_train.iloc[i, 0] = re.sub(r'\sו', ' ', df_train.iloc[i, 0])
    # replace multiple spaces with single space
    df_train.iloc[i, 0] = re.sub(r'\s+', ' ', df_train.iloc[i, 0], flags=re.I)
    
df_train.tail(5)

,story,gender
359,לפני שנים התגוררתי למשך שנה בגרמניה מטרת המעבר...,m
360,ביום הבחירות נסענו לבקר את אימי זל בבית הקברות...,m
361,בשנה אחרונה חוויתי לראשונה את תהליך חיפוש העבו...,m
362,אני סטודנט במכללה בסמסטר בשנת תשפא נגשתי למבחן...,m
363,הייתי מדריכה בכפר נוער מתאם הכפר היינו צריכים...,f


In [7]:
for i in df_train.index:
    # replace important words that ends with תי with other words
    df_train.iloc[i, 0] = str(df_train.iloc[i, 0]).replace(' אשתי ', ' אישה שלי ')
    df_train.iloc[i, 0] = str(df_train.iloc[i, 0]).replace(' אישתי ', ' אישה שלי ')
    df_train.iloc[i, 0] = str(df_train.iloc[i, 0]).replace(' זוגתי ', ' בת זוג שלי ')
    df_train.iloc[i, 0] = str(df_train.iloc[i, 0]).replace(' ארוסתי ', ' ארוסה שלי ')
    df_train.iloc[i, 0] = str(df_train.iloc[i, 0]).replace(' חברתי ', ' חברה שלי ')
    df_train.iloc[i, 0] = str(df_train.iloc[i, 0]).replace(' חברותי ', ' חברות שלי ')
    df_train.iloc[i, 0] = str(df_train.iloc[i, 0]).replace(' שתי ', ' שתיים ')
    # remove all words that ends with תי, for examle: הייתי, אכלתי, הלכתי, ראיתי etc.
    df_train.iloc[i, 0] = re.sub(r'[א-ת]+תי\s', '', df_train.iloc[i, 0])

df_train.head(5)

,story,gender
0,בוקר אחד סהרורי מהמיטה קצת מטושטש בצעדים כבדים...,m
1,לחבר שלי היה יום הולדת חיפשנו מה אפשר לעשות לו...,m
2,השנה האחרונה הייתה שנת קורונה שנה לא פשוטה בקנ...,m
3,לפני כחצי שנה לגור בצפון עם בת בת זוג שלי עברנ...,m
4,יום חמישי רגיל תמיד מתחיל לעבור טיפה מאוחר יות...,m


In [8]:
# Finding the most common words (and add them to the stop words):
words = []

for i in df_train.index:
    split_words = str(df_train.iloc[i, 0]).split()
    words = words + split_words

counter = Counter(words)
common_words = [key for key, _ in counter.most_common(200)]

In [9]:
# This two words decreased the accuracy:
good_words = ['בת', 'שהיא']

for word in good_words:
    common_words.remove(word)

len(common_words)

198

In [10]:
# Stop words:

stop_words = ['אני','אתה','אנחנו','הם','שלי','שלך','שלנו','שלכם','שלהם','שלהן','לי','לנו','לכם','להם','זה','זאת',
'אלה','אלו','תחת','תחחת' 'מתחת','מעל','בין','עם','עד','על','אל','מול','של','אצל','כמו','אחר','אותו','בלי','לפני','אחרי','מאחורי','עלי','עליך',
'עלינו','עליכם','עליכן','עליהם','עליהן','כל','כולם','כך','ככה','כזה','אותי','אותם','אותך','אותנו','אתכם','אתכן','איתי','איתו','איתך',
'איתם','איתנו','איתכם','איתכן','יהיה','הייתי','היתי','היתה','הייתה','היה','להיות','עצמי','עצמם','עצמן','עצמנו','עצמהם','עצמהן','מי','מה',
'איפה','היכן','אם','לאן','איזה','מהיכן','איך','כיצד','מתי','כאשר','למרות','למה','מדוע','כי','יש','אין','אך','מנין','מאין','מאיפה','יכלו',
'יוכלו','לא','רק','אולי','לאו','אי','נגד','אף','מצד','בשביל','לבין','באמצע','בתוך','דרך','מבעד','באמצעות','למעלה','למטה','מחוץ',
'מן','מין','לעבר','מכאן','כאן','הנה','הרי','פה','שם','שוב','אבל','מבלי','מלבד','בלבד','בגלל','מכיוון','אשר','ואילו','כפי','אז','כן',
'לפיכך','מאד','מעט','מעטים','במידה','יותר','מדי','גם','נו','אחרת','אחרים','או','לגמריי','לגמרי','תמיד','היום','מחר','אתמול','שבוע','שנה',
'חודש','בנוסף','מעבר','כשהייתי','אילו','לעומת','בעוד','עוד','בניגוד','ככל','מפני','בשל','עקב','בזכות','בכדי','כדי','כדאי','למען',
'אכן','אמנם','משמע','כזכור','כבר','בקרוב','לעולם','מעולם','טרם','כמובן','ללא','זמן','טיסה','חול','שום','מכן','קרה','יקרה','קורה','פשוט',
'קל','עי','עפ','שעות','שעה','שאר','יום','ימים','חודשים','שנים','קורונה','חולה','חולים','בכל','אפילו','עדיין','למעשה',
'בעצם','למחרת','בבוקר','בלילה','בוקר','לילה','בערב','ערב','רואה','רוצה','עושה','הרבה','מצפה','קונה','מצידי','בטח','יתכן','ממש','בזה']

In [11]:
words = []
words = words + common_words + stop_words
words = list(set(words))   # without duplicates

### Function that measures the avarage f1 score:
similar to f1 macro

In [12]:
def f1_avg_calc(tag, predicted):
    conf_mat = metrics.confusion_matrix(tag, predicted)

    TP = conf_mat[0][0]
    FP = conf_mat[0][1]
    FN = conf_mat[1][0]
    TN = conf_mat[1][1]

    precision_f = TP / (TP + FP)
    recall_f = TP / (TP + FN)

    precision_m = TN / (TN + FN)
    recall_m = TN / (TN + FP)

    F1_f = 2 * (precision_f * recall_f) / (precision_f + recall_f)
    F1_m = 2 * (precision_m * recall_m) / (precision_m + recall_m)

    F1_avg = (F1_f + F1_m) / 2
    return F1_avg

### Splitting the train into train and test (validation set) for 80:20

In [13]:
X_train_v, X_test_v, y_train_v, y_test_v = train_test_split(df_train.story, df_train.gender, test_size=0.2, random_state=0)

### Finding the best algorithm:
We want to find an algorithm which returns a close f1 score for both the valid set and the test set.
#### NB gives the best results.
I tried different hyperparameters for each algorithm.

In [15]:
names = ["Nearest Neighbors", "Linear SVM", "Decision Tree",
         "Neural Net", "AdaBoost", "Naive Bayes"]

classifiers = [
    KNeighborsClassifier(7),
    SVC(kernel="linear"),
    DecisionTreeClassifier(max_depth=15),
    MLPClassifier(alpha=0.0001, max_iter=1000),
    AdaBoostClassifier(),
    MultinomialNB()]

for name, clf in zip(names, classifiers):
    name
    
    text_clf_train = Pipeline([('vect', CountVectorizer(max_features=2000)), ('clf', clf)])
    text_clf_test = Pipeline([('vect', CountVectorizer(max_features=2000)), ('clf', clf)])

    text_clf_train.fit(X_train_v, y_train_v)
    predicted_train_v = text_clf_train.predict(X_test_v)
    
    text_clf_test.fit(df_train.story, df_train.gender)
    predicted_test = text_clf_test.predict(df_test.story)
    
    print("for train:")
    f1_avg_calc(y_test_v, predicted_train_v)
    
    print("for test:")
    f1_avg_calc(my_test.tag, predicted_test)
    
    print('\n\n')

'Nearest Neighbors'

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=2000, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=7, p=2,
                                      weights='uniform'))],
         verbose=False)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=2000, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=7, p=2,
                                      weights='uniform'))],
         verbose=False)

for train:


0.45811240721102864

...

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=2000, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

for train:


0.7482758620689656

for test:


0.724398416082851

#### the best parameters I found gave me avg_f1 = 0.7674

In [16]:
text_clf_nb = Pipeline([
    ('vect', CountVectorizer(stop_words=words, ngram_range=(1,6), max_features=4500)),
    ('clf', MultinomialNB())])

text_clf_nb.fit(df_train.story, df_train.gender)
predicted_test = text_clf_nb.predict(df_test.story)

print("f1-score:")
f1_avg_calc(my_test.tag, predicted_test)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=4500, min_df=1,
                                 ngram_range=(1, 6), preprocessor=None,
                                 stop_words=['כך', 'שלנו', 'למעלה', 'קרה',
                                             'עליהן', 'להם', 'אחת', 'אין',
                                             'אבל', 'לאחר', 'טרם', 'איתך',
                                             'מבלי', 'אתכם', 'אוכל', 'פתאום',
                                             'אנשים', 'השנה', 'מפני', 'שעה',
                                             'אכן', 'שבוע', 'אחרת', 'מחוץ',
                                             'דבר', 'שלו', 'אמר', 'כשהייתי',
        

f1-score:


0.767400250492038

### Save output to csv

In [18]:
df_predicted = df_test.copy()

df_predicted.drop(labels='story', axis=1, inplace=True)
df_predicted['predicted_category'] = predicted_test

df_predicted.head(5)

,test_example_id,predicted_category
0,0,m
1,1,f
2,2,m
3,3,f
4,4,m


In [ ]:
df_predicted.to_csv('classification_results.csv',index=False)